# Build Slice

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType

ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()
    
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")
print(os.environ['FABRIC_CREDMGR_HOST'])

slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='all', scope='all')

# Initialize the slice manager
slice_manager.initialize()

slice_name='InstrumentizeTest'
site='TACC'
image_name='default_ubuntu_20'

# Create topology
myExperiment = ExperimentTopology()

# Add nod
Meas_Node = myExperiment.add_node(name="Meas_Node", site=site)
Node1 = myExperiment.add_node(name="Node1", site=site)
Node2 = myExperiment.add_node(name="Node2", site=site)


    
# Set capacities

cap = Capacities()
cap.set_fields(core=2, ram=16, disk=100)

# Set Properties
Meas_Node.set_properties(capacities=cap, image_type='qcow2', image_ref=image_name)
Node1.set_properties(capacities=cap, image_type='qcow2', image_ref=image_name)
Node2.set_properties(capacities=cap, image_type='qcow2', image_ref=image_name)

Meas_Node.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_5, name='MDCnic1')
Node1.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_5, name='MDCnic2')
#Node1.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_5, name='MDCnic3')
#Node2.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_5, name='MDCnic4')
Node2.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_5, name='MDCnic5')



print(myExperiment.interface_list)
# Specify VLAN
inter = myExperiment.interface_list[0]
label = inter.get_property(pname="labels")
label.vlan = "200"
inter.set_properties(labels = label)

inter = myExperiment.interface_list[1]
label = inter.get_property(pname="labels")
label.vlan = "201"
inter.set_properties(labels = label)

inter = myExperiment.interface_list[2]
label = inter.get_property(pname="labels")
label.vlan = "200"
inter.set_properties(labels = label)

inter = myExperiment.interface_list[3]
label = inter.get_property(pname="labels")
label.vlan = "202"
inter.set_properties(labels = label)

inter = myExperiment.interface_list[4]
label = inter.get_property(pname="labels")
label.vlan = "201"
inter.set_properties(labels = label)

inter = myExperiment.interface_list[5]
label = inter.get_property(pname="labels")
label.vlan = "202"
inter.set_properties(labels = label)

# L2PTP Service
myExperiment.add_network_service(name='ptp1', nstype=ServiceType.L2PTP, interfaces=[Meas_Node.interface_list[0], Node1.interface_list[0]])
myExperiment.add_network_service(name='ptp2', nstype=ServiceType.L2PTP, interfaces=[Meas_Node.interface_list[1], Node2.interface_list[0]])
myExperiment.add_network_service(name='ptp3', nstype=ServiceType.L2PTP, interfaces=[Node1.interface_list[1], Node2.interface_list[1]])
    
# Generate Slice Graph
slice_graph = myExperiment.serialize()
# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
else:
    print(f"Failure: {reservations}")

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json
status, slices = slice_manager.slices(excludes=[SliceState.Closing, SliceState.Dead])

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))
    
    slice_object=list(filter(lambda s: s.slice_name == slice_name, slices))[0]
else:
    print(f"Failure: {slices}")

# Only run after slice stableOK:

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])
slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
print(slice)
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)
print(return_status)
for node_name, node in experiment_topology.nodes.items():
    
    print("Node:")
    print("   Name              : {}".format(node.name))
    print("   Cores             : {}".format(node.get_property(pname='capacity_allocations').core))
    print("   RAM               : {}".format(node.get_property(pname='capacity_allocations').ram))
    print("   Disk              : {}".format(node.get_property(pname='capacity_allocations').disk))
    print("   Image             : {}".format(node.image_ref))
    print("   Image Type        : {}".format(node.image_type))
    print("   Host              : {}".format(node.get_property(pname='label_allocations').instance_parent))
    print("   Site              : {}".format(node.site))
    print("   Management IP     : {}".format(node.management_ip))
    print("   Reservation ID    : {}".format(node.get_property(pname='reservation_info').reservation_id))
    print("   Reservation State : {}".format(node.get_property(pname='reservation_info').reservation_state))
    print("   Components        : {}".format(node.components))
    print("   Interfaces        : {}".format(node.interfaces))
    print(node)    

# Run on Nodes:
## Node0:
```
ip link add link ens7 name ens7.200 type vlan id 200
ip link set dev ens7.200 up
ip addr add 192.168.10.52/24 dev ens7.200
```
```
ip link add link ens8 name ens8.201 type vlan id 201
ip link set dev ens8.201 up
ip addr add 192.168.10.53/24 dev ens8.201
```
## Node1:
```
ip link add link ens7 name ens7.200 type vlan id 200
ip link set dev ens7.200 up
ip addr add 192.168.10.51/24 dev ens7.200
```
```
ip link add link ens8 name ens8.200 type vlan id 202
ip link set dev ens8.202 up
ip addr add 192.168.10.53/24 dev ens8
```
## Node2:
```
ip link add link ens7 name ens7.201 type vlan id 201
ip link set dev ens7.201 up
ip addr add 192.168.10.51/24 dev ens7.201
```
```
ip link add link ens8 name ens8.202 type vlan id 202
ip link set dev ens8.202 up
ip addr add 192.168.10.52/24 dev ens8
```

# Instrumentize Slice

# Need to add Python code to do the following
git clone git@github.com:fabric-testbed/MeasurementFramework.git
git checkout add_fabric_experiment_role




# Delete Slice

In [10]:
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received None
